# Week 3: Webscraping
Today, we will go through scrap multiple pages, try out selenium, google news! (Make sure to have installed: selenium & pygooglenews)

Let's look at the website first. <a href="http://quotes.toscrape.com/"> Quotes! </a>

In [259]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

quote_list = []
author_list = []

for i in range(1, 11): # from 1 to 10
    url = f'http://quotes.toscrape.com/page/{i}/'
    response = requests.get(url)
    html = response.text
    soup = bs(html, 'html.parser')

    quotes = soup.find_all('span', itemprop='text')
    authors = soup.find_all('small', class_='author')

    for quote in quotes:
        quote_list.append(quote.get_text())

    for author in authors:
        author_list.append(author.get_text())

    df1 = pd.DataFrame(data=quote_list, columns=['Quotes'])
    df2 = pd.DataFrame(data=author_list, columns=['Authors'])
    quotes = pd.concat([df1, df2], axis=1)
    quotes.to_excel('quotes_week2.xlsx')

# Selenium!

In [260]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [261]:
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/c/brettconti/videos')
# time.sleep(5) # because the page might need some time to load
driver.implicitly_wait(10)  # kinda the same as the line before, but if the loading is done it will continue (not like time.sleep()
videos = driver.find_elements(By.CLASS_NAME, "style-scope ytd-grid-video-renderer")
# specify a class name that occurs multiple times and use find_elements to get a list returned

video_list = []
for video in videos:
    title = video.find_element(By.XPATH, './/*[@id="video-title"]').text  # put a dot at front!
    views = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[1]').text
    posted = video.find_element(By.XPATH, './/*[@id="metadata-line"]/span[2]').text
    video_list.append({'title': title, 'views': views, 'posted': posted})

df = pd.DataFrame(video_list)

In [262]:
driver.quit()  # just closes the tab -> always do that!

# Google News!

In [263]:
from pygooglenews import GoogleNews
gn = GoogleNews()

In [264]:
top = gn.top_news()

In [265]:
business = gn.topic_headlines('business')

In [266]:
search = gn.search('%22New+York%22')  #, when = '6m')

In [267]:
print(search.keys())

dict_keys(['feed', 'entries'])


In [268]:
print(search['feed'])

{'generator_detail': {'name': 'NFE/5.0'}, 'generator': 'NFE/5.0', 'title': '"%22New+York%22" - Google News', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': '"%22New+York%22" - Google News'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/search?q=%2522New%2BYork%2522&ceid=US:en&hl=en-US&gl=US'}], 'link': 'https://news.google.com/search?q=%2522New%2BYork%2522&ceid=US:en&hl=en-US&gl=US', 'language': 'en-US', 'publisher': 'news-webmaster@google.com', 'publisher_detail': {'email': 'news-webmaster@google.com'}, 'rights': '2022 Google Inc.', 'rights_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': '2022 Google Inc.'}, 'updated': 'Fri, 14 Oct 2022 07:46:25 GMT', 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=10, tm_mday=14, tm_hour=7, tm_min=46, tm_sec=25, tm_wday=4, tm_yday=287, tm_isdst=0), 'subtitle': 'Google News', 'subtitle_detail': {'type': 'text/html', 'language': None, 'base': '', 'valu

In [269]:
print(search['entries'])

[{'title': "10/12/22: New York's DEC chief in Ukraine | NCPR News - North Country Public Radio", 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': "10/12/22: New York's DEC chief in Ukraine | NCPR News - North Country Public Radio"}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.northcountrypublicradio.org/news/story/46712/20221012/10-12-22-new-york-s-dec-chief-in-ukraine'}], 'link': 'https://www.northcountrypublicradio.org/news/story/46712/20221012/10-12-22-new-york-s-dec-chief-in-ukraine', 'id': 'CBMiamh0dHBzOi8vd3d3Lm5vcnRoY291bnRyeXB1YmxpY3JhZGlvLm9yZy9uZXdzL3N0b3J5LzQ2NzEyLzIwMjIxMDEyLzEwLTEyLTIyLW5ldy15b3JrLXMtZGVjLWNoaWVmLWluLXVrcmFpbmXSAQA', 'guidislink': False, 'published': 'Wed, 12 Oct 2022 20:01:52 GMT', 'published_parsed': time.struct_time(tm_year=2022, tm_mon=10, tm_mday=12, tm_hour=20, tm_min=1, tm_sec=52, tm_wday=2, tm_yday=285, tm_isdst=0), 'summary': '<a href="https://www.northcountrypublicradio.org/news/story/4671

In [270]:
for item in search['entries']:
    print(item['title'])

10/12/22: New York's DEC chief in Ukraine | NCPR News - North Country Public Radio
Volleyball: Wolverines claim conference title over Eagles - Wadena Pioneer Journal
New York: State Regulators Finalize Home Cultivation Rules for Qualified Patients - Norml
New York City’s incredible shrinking starter home market - The Real Deal
Miami Dolphins News 10/10/22: Jets Dominate Dolphins 40-17 - The Phinsider
Where the best fall leaves will be in New York this year - WXXI News
Over $1.7 million allocated from NYS Budget to upgrade some CNY libraries - The New York State Senate
PODCAST: Giants are winning games, but is it sustainable? - Giants Wire
Photos from the 'Terrifier 2' Red Carpet Premiere in New York! - Bloody Disgusting
Russia's Alrosa Discovers 22 New Diamond Deposits in Zimbabwe - Bloomberg
IMB trustees appoint 22 new missionaries, approve $282.7M budget - The Christian Index
Rookie Camp Roundup, Analysis & Observations - New York Islanders Hockey Now - New York Hockey Now
New York C

In [271]:
def get_title(search):
    stories = []
    search = gn.search(search)
    for item in search['entries']:
        stories.append({'title': item.title, 'link': item.link})
    return stories

In [272]:
a = get_title('weather')

In [273]:
weather = pd.DataFrame(a)

In [274]:
weather

,title,link
0,Unusually cold autumn weather to blast eastern...,https://www.washingtonpost.com/climate-environ...
1,First Alert Weather: Rain likely for the Phoen...,https://www.azfamily.com/2022/10/13/first-aler...
2,La Nina winter will mean parts of US will feel...,https://www.foxweather.com/extreme-weather/la-...
3,"N.J. weather: Torrential rain, damaging winds ...",https://www.nj.com/weather/2022/10/nj-weather-...
4,Parts of Mid-South under Fire Weather Watch. W...,https://wreg.com/news/local/parts-of-mid-south...
...,...,...
95,"Orlando Weather Forecast: Cooler, drier air fo...",https://www.fox35orlando.com/news/orlando-weat...
96,‘Now is the time to prepare’: Florida resident...,https://www.foxweather.com/extreme-weather/flo...
97,Governor Newsom Takes Action to Support Commun...,https://www.gov.ca.gov/2022/09/16/governor-new...
98,"September 22, PM Weather Forecast Update - 50-...",https://www.wlfi.com/news/september-22-pm-weat...


In [275]:
weather['Title'] = weather['title'].str.split(' - ', expand=True)[0]

In [276]:
weather['Source'] = weather['title'].str.split(' - ', expand=True)[1]

In [277]:
weather.drop(['title'], axis=1)

,link,Title,Source
0,https://www.washingtonpost.com/climate-environ...,Unusually cold autumn weather to blast eastern...,The Washington Post
1,https://www.azfamily.com/2022/10/13/first-aler...,First Alert Weather: Rain likely for the Phoen...,Arizona's Family
2,https://www.foxweather.com/extreme-weather/la-...,La Nina winter will mean parts of US will feel...,Fox Weather
3,https://www.nj.com/weather/2022/10/nj-weather-...,"N.J. weather: Torrential rain, damaging winds ...",NJ.com
4,https://wreg.com/news/local/parts-of-mid-south...,Parts of Mid-South under Fire Weather Watch. W...,WREG NewsChannel 3
...,...,...,...
95,https://www.fox35orlando.com/news/orlando-weat...,"Orlando Weather Forecast: Cooler, drier air fo...",FOX 35 Orlando
96,https://www.foxweather.com/extreme-weather/flo...,‘Now is the time to prepare’: Florida resident...,Fox Weather
97,https://www.gov.ca.gov/2022/09/16/governor-new...,Governor Newsom Takes Action to Support Commun...,Office of Governor Gavin Newsom
98,https://www.wlfi.com/news/september-22-pm-weat...,"September 22, PM Weather Forecast Update",50-Degree Temperature Drop From Yesterday to F...


In [278]:
weather.rename(columns={'link': 'Link'}, inplace=True)

Let's reorder

In [279]:
weather = weather[['Title', 'Source', 'Link']]

In [280]:
weather.to_excel('weather.xlsx')